In [18]:
import pandas as pd
import numpy as np

import pyspark.sql.functions as F
#from pyspark.sql.functions import*
from pyspark.sql.window import Window

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

import torch
from sklearn.preprocessing import StandardScaler

In [ ]:
#df_admissions1 = pd.read_csv('C:/Users/Chicki/Desktop/Data/admissions_new1.csv')
#df_admissions.count()
#df_admissions1[df_admissions1['HADM_ID'] == 143363]

In [ ]:
df_admissions = spark.read.csv('C:/Users/Chicki/Desktop/Data/admissions_new1.csv')
#df_admissions.show()
#df_admissions = pd.read_csv('C:/Users/Chicki/Desktop/Data/admissions_new1.csv')
#print (df_admissions.head(5))
#query_admissions = df_admissions.query('HADM_ID == 1657660')
#print(query_admissions)
#df_admissions.sort_values(by=['SUBJECT_ID']).head(20)


In [ ]:
w = Window.partitionBy('SUBJECT_ID').orderBy('HADM_ID')
#df_admissions_reduced = df_admissions.withColumn('Rank', dense_rank().over(w))
#df_admissions_reduced.filter(df.Rank == 1).head()
#df_admissions_reduced = df_admissions.withColumn("row_number",F.row_number().over(Window.partitionBy(df_admissions._c1).orderBy(df_admissions._c2.desc()))).filter(F.col("row_number")==1).drop("row_number")
df_admissions_reduced1 = df_admissions.withColumn("row_number",F.row_number().over(Window.partitionBy(df_admissions._c1).orderBy(df_admissions._c2.desc()))).filter(F.col("row_number")==1).drop("row_number")

In [ ]:
df_admissions_reduced1.count()

In [ ]:
#df_admissions_reduced1.withColumnRenamed("_c1","SUBJECT_ID").show()
df_admissions_reduced1.withColumnRenamed("_c2","HADM_ID").show()


In [ ]:
## IF using the FULL admissions file, use this one...
df_admissions3 = df_admissions_reduced1.toPandas()
df_admissions3.count()

In [ ]:
#df_admissions2 = df_admissions_reduced1.limit(30000).toPandas()
#df_admissions2.head(10)
#df_admissions2.count()
df_admissions2.dtypes

In [15]:
df_patients = pd.read_csv('C:/Users/Chicki/Desktop/Data/patients.csv')
#print(df_patients.query('SUBJECT_ID == 36'))
print(df_patients['DOB'].unique())

['2075-03-13 00:00:00' '2164-12-27 00:00:00' '2090-03-15 00:00:00' ...
 '2086-03-29 00:00:00' '2026-05-25 00:00:00' '2076-07-25 00:00:00']


In [16]:
df_patients.head()

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0
3,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
4,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0


In [ ]:
#df_icu_stays = pd.read_csv('C:/Users/Chicki/Desktop/Data/ICUSTAYS1.csv')
#df_icu_stays.count()
#print ((df_icu_stays.head(5)))
#df_icu_stays.sort_values(by=['SUBJECT_ID']).head(20)


In [ ]:
df_chart_events = pd.read_csv('C:/Users/Chicki/Desktop/Data/CHARTEVENTS.csv', nrows=50000000, low_memory = False)
#df_chart_events.count()
#df_chart_events1 = pd.read_csv('C:/Users/Chicki/Desktop/Data/CHARTEVENTS.csv', nrows=10000000)

## REMOVE the ERROR records 
#df_chart_events1 = df_chart_events1[df_chart_events1["ERROR"] == 0]
#df_chart_events1.count()

#df_chart_events = df_chart_events[df_chart_events["ERROR"] == 0]
#df_chart_events.count()


#df_chart_events.cache()

#df_chart_events1.count()
#df_chart_events.count()
#print((df_chart_events.query('WARNING == 1').head(5)))
#print((df_chart_events.query('ITEMID < 10884').head(5)))


In [ ]:
df_chart_events.count()

In [ ]:
#print((df_chart_events1.query('ERROR == 1').count()))
#print((df_chart_events1.query('WARNING == 1').tail()))
#df_chart_events.isna().sum()
#print(df_chart_events1['WARNING'].unique())
#print(df_chart_events1['RESULTSTATUS'].unique())

# Remove all unnecessary columns from Chart Events

In [ ]:
## REMOVE RECORDS with NA in admission id
df_chart_events = df_chart_events.dropna(subset=['HADM_ID'])


## REMOVE the unnecessary columns
df_chart_events = df_chart_events.drop(['ROW_ID','CHARTTIME','STORETIME','CGID','STOPPED','ERROR', 'RESULTSTATUS', 'WARNING', 'ICUSTAY_ID'], axis=1)
#df_chart_events.count()
#print((df_chart_events1.head(5)))

In [ ]:
### HAVE ONLY THE REDUCED RECORDS FROM ADMISSIONS BEING USED HERE IN LAB EVENTS

## Use this to have the limited list of admissions
admissions_list = df_admissions2['_c2'].astype('int').tolist()


## Use this for the full list of admissions and not the limited one
#admissions_list = df_admissions3['_c2'].astype('str').astype('int').tolist()
#admissions_list = df_admissions3['_c2'].tolist()
#admissions_list = np.int_(df_admissions3['_c2']).tolist()
print(admissions_list[0:10])

df_chart_events = df_chart_events[df_chart_events.HADM_ID.isin(admissions_list)]
df_chart_events.count()

# Remove all unnecessary data/columns from Lab Events

In [ ]:
df_lab_events = pd.read_csv('C:/Users/Chicki/Desktop/Data/LABEVENTS.csv')
#df_lab_events = pd.read_csv('C:/Users/Chicki/Desktop/Data/LABEVENTS.csv', nrows=10000000)
#print(df_lab_events.head(5))
df_lab_events.count()
## REMOVE the error records from the lab events
#df_lab_events = df_lab_events.drop(['ROW_ID', 'CHARTTIME'], axis = 1)
#df_lab_events.tail()
#df_lab_events.isna().sum()

In [ ]:
print(df_lab_events.query('HADM_ID == 165660').head())

In [ ]:
### HAVE ONLY THE REDUCED RECORDS FROM ADMISSIONS BEING USED HERE IN LAB EVENTS for selection
admissions_list = df_admissions2['_c2'].astype('int').tolist()
#print(admissions_list)

df_lab_events1 = df_lab_events[df_lab_events.HADM_ID.isin(admissions_list)]
df_lab_events1.count()

In [ ]:
## ONLY have the requisite ITEM details we are looking for in the LAB & Chart events
items_list = [223761, 220045, 220046, 220047, 220179, 220180, 220181, 220052, 223835, 224161, 221906, 225411, 226260, 50821, 223835, 1884, 50931, 50882, 50912, 51221, 51222, 51265, 50971, 51006, 51301, 51009, 50865, 50929, 50997, 4478, 8125, 5982, 8157, 8204, 6287, 6720, 6807, 3112, 225411, 6185]
print(items_list)


In [ ]:
#df_lab_events1 = df_lab_events1[df_lab_events1.ITEMID.isin(items_list)]
df_chart_events = df_chart_events[df_chart_events.ITEMID.isin(items_list)]
#df_lab_events1.count()

In [ ]:
df_chart_events.count()

In [ ]:
#query_lab_events = df_lab_events.query('FLAG == "abnormal"')
#query_lab_events = df_lab_events.query('LABEL.str.contains("bun", na=False)', engine = 'python')
#query_lab_events.tail(10)

## REMOVE RECORDS with NA in admission id
df_lab_events1 = df_lab_events1.dropna(subset=['HADM_ID', 'ITEMID', 'VALUE', 'SUBJECT_ID'])
df_lab_events1.count()

# Create the new columns on Chart Events

In [ ]:
#df_chart_events1 = df_chart_events1.drop(['temphigh'], axis = 1)
df_chart_events.count()

In [ ]:
## ASSIGN temphigh column based on values 
df_chart_events['temphigh'] = ((df_chart_events['ITEMID']== 223761) & (df_chart_events['VALUENUM'] > 99).astype('int'))

In [ ]:
## ASSIGN heartratehigh column based on values
df_chart_events['heartratehigh'] = (((df_chart_events['ITEMID'] == 220045) | (df_chart_events['ITEMID'] == 220046) | (df_chart_events['ITEMID'] == 220047)) & (df_chart_events['VALUENUM'] > 100))

In [ ]:
## ASSIGN sysbplow column based on values
df_chart_events['sysbplow'] = ((df_chart_events['ITEMID'] == 220179) & (df_chart_events['VALUENUM'] < 90).astype('int'))

In [ ]:
#print(df_chart_events.query('ITEMID == 220179 and VALUENUM < 90').head())

In [ ]:
print(df_chart_events.query('ITEMID in (220045, 220046, 220047) & VALUENUM < 100').tail(5))

In [ ]:
#df_chart_events1.head()
print(df_chart_events.query('ITEMID == 223761 and VALUE > 99').head())

In [ ]:
## ASSIGN diasbplow column based on values
df_chart_events['diasbplow'] = ((df_chart_events['ITEMID'] == 220180) & (df_chart_events['VALUENUM'] < 60).astype('int'))

In [ ]:
print(df_chart_events.query('ITEMID == 220180 and VALUE > 60').head())

In [ ]:
## ASSIGN meanbplow column based on values
df_chart_events['meanbplow'] = ((df_chart_events['ITEMID'] == 220181) & (df_chart_events['VALUENUM'] < 70).astype('int'))

In [ ]:
print(df_chart_events.query('ITEMID == 220181 and VALUE > 70').head())

In [ ]:
## ASSIGN fio2high column based on values
df_chart_events['fio2high'] = ((df_chart_events['ITEMID'] == 223835) & (df_chart_events['VALUENUM'] > 50).astype('int'))

In [ ]:
print(df_chart_events.query('ITEMID == 224161').head(5))

In [ ]:
## ASSIGN respratehigh column based on values
df_chart_events['respratehigh'] = ((df_chart_events['ITEMID'] == 224161) & (df_chart_events['VALUENUM'] > 20).astype('int'))

In [ ]:
print(df_chart_events.query('ITEMID == 224161').head())

In [ ]:
## ASSIGN norepinephrine column based on values
df_chart_events['norepinephrine'] = (((df_chart_events['ITEMID'] == 3112) | (df_chart_events['ITEMID'] == 221906)) & (df_chart_events['VALUENUM'] > 0))

In [ ]:
print(df_chart_events.query('ITEMID == 221906 | ITEMID == 3112 | ITEMID == 224161').head())

In [ ]:
## ASSIGN mechanical_ventilator column based on values
df_chart_events['mechanical_ventilator'] = (((df_chart_events['ITEMID'] == 6185) | (df_chart_events['ITEMID'] == 226260) | (df_chart_events['ITEMID'] == 225411)))

In [ ]:
#print(df_chart_events1.query('ITEMID == 226260 | ITEMID == 6185 | ITEMID == 225411').head())
df_chart_events.head()

# Create the new columns on Lab Events

In [ ]:
## ASSIGN SpO2 column based on values
df_lab_events1['spo2high'] = ((df_lab_events1['ITEMID'] == 50821) & (df_lab_events1['VALUENUM'] > 100).astype('int'))

In [ ]:
print(df_lab_events1.query('ITEMID == 50821').head())
#print(df_lab_events1.head())

In [ ]:
## ASSIGN glucose column based on values
df_lab_events1['glucoselow'] = ((df_lab_events1['ITEMID'] == 50931) & (df_lab_events1['VALUENUM'] < 75).astype('int'))

In [ ]:
print(df_lab_events1.query('ITEMID == 50931').tail())

In [ ]:
## ASSIGN bicarbonatehigh column based on values
df_lab_events1['bicarbonatehigh'] = ((df_lab_events1['ITEMID'] == 50882) & (df_lab_events1['VALUENUM'] > 29).astype('int'))

In [ ]:
print(df_lab_events1.query('ITEMID == 50882').tail())

In [ ]:
## ASSIGN bicarbonatelow column based on values
df_lab_events1['bicarbonatelow'] = ((df_lab_events1['ITEMID'] == 50882) & (df_lab_events1['VALUENUM'] < 22).astype('int'))

In [ ]:
print(df_lab_events1.query('ITEMID == 50882').tail())

In [ ]:
## ASSIGN creatininehigh column based on values
df_lab_events1['creatininehigh'] = ((df_lab_events1['ITEMID'] == 50912) & (df_lab_events1['VALUENUM'] > 1.3).astype('int'))

In [ ]:
print(df_lab_events1.query('ITEMID == 50912').tail())

In [ ]:
## ASSIGN creatininelow column based on values
df_lab_events1['creatininelow'] = ((df_lab_events1['ITEMID'] == 50912) & (df_lab_events1['VALUENUM'] < 0.6).astype('int'))

In [ ]:
print(df_lab_events1.query('ITEMID == 50912').tail())

In [ ]:
## ASSIGN hematocrithigh column based on values
df_lab_events1['hematocrithigh'] = ((df_lab_events1['ITEMID'] == 51221) & (df_lab_events1['VALUENUM'] > 48).astype('int'))

In [ ]:
print(df_lab_events1.query('ITEMID == 51221').tail(20))

In [ ]:
## ASSIGN hematocritlow column based on values
df_lab_events1['hematocritlow'] = ((df_lab_events1['ITEMID'] == 51221) & (df_lab_events1['VALUENUM'] < 36).astype('int'))

In [ ]:
print(df_lab_events1.query('ITEMID == 51221').tail(20))

In [ ]:
## ASSIGN hemoglobinhigh column based on values
df_lab_events1['hemoglobinhigh'] = ((df_lab_events1['ITEMID'] == 51222) & (df_lab_events1['VALUENUM'] > 16).astype('int'))

In [ ]:
print(df_lab_events1.query('ITEMID == 51222').head(20))

In [ ]:
## ASSIGN hemoglobinlow column based on values
df_lab_events1['hemoglobinlow'] = ((df_lab_events1['ITEMID'] == 51222) & (df_lab_events1['VALUENUM'] < 11.6).astype('int'))

In [ ]:
print(df_lab_events1.query('ITEMID == 51222').head(20))

In [ ]:
## ASSIGN platelethigh column based on values
df_lab_events1['platelethigh'] = ((df_lab_events1['ITEMID'] == 51265) & (df_lab_events1['VALUENUM'] > 450).astype('int'))

In [ ]:
print(df_lab_events1.query('ITEMID == 51265').head(20))

In [ ]:
## ASSIGN plateletlow column based on values
df_lab_events1['plateletlow'] = ((df_lab_events1['ITEMID'] == 51265) & (df_lab_events1['VALUENUM'] < 150).astype('int'))

In [ ]:
print(df_lab_events1.query('ITEMID == 51265').head(20))

In [ ]:
## ASSIGN potassiumhigh column based on values
df_lab_events1['potassiumhigh'] = ((df_lab_events1['ITEMID'] == 50971) & (df_lab_events1['VALUENUM'] > 5.2).astype('int'))

In [ ]:
print(df_lab_events1.query('ITEMID == 50971').head(20))

In [ ]:
## ASSIGN potassiumlow column based on values
df_lab_events1['potassiumlow'] = ((df_lab_events1['ITEMID'] == 50971) & (df_lab_events1['VALUENUM'] < 3.6).astype('int'))

In [ ]:
print(df_lab_events1.query('ITEMID == 50971').head(20))

In [ ]:
## ASSIGN bunhigh column based on values
df_lab_events1['bunhigh'] = ((df_lab_events1['ITEMID'] == 51006) & (df_lab_events1['VALUENUM'] > 23).astype('int'))

In [ ]:
print(df_lab_events1.query('ITEMID == 51006').head(20))

In [ ]:
## ASSIGN bunlow column based on values
df_lab_events1['bunlow'] = ((df_lab_events1['ITEMID'] == 51006) & (df_lab_events1['VALUENUM'] < 7).astype('int'))

In [ ]:
print(df_lab_events1.query('ITEMID == 51006').tail(20))

In [ ]:
## ASSIGN wbchigh column based on values
df_lab_events1['wbchigh'] = ((df_lab_events1['ITEMID'] == 51301) & (df_lab_events1['VALUENUM'] > 11).astype('int'))

In [ ]:
print(df_lab_events1.query('ITEMID == 51301').tail(20))

In [ ]:
## ASSIGN wbclow column based on values
df_lab_events1['wbclow'] = ((df_lab_events1['ITEMID'] == 51301) & (df_lab_events1['VALUENUM'] < 4.5).astype('int'))

In [ ]:
print(df_lab_events1.query('ITEMID == 51301').head(20))

In [ ]:
## ASSIGN antibiotics column based on values
df_lab_events1['antibiotics'] = (((df_lab_events1['ITEMID'] == 51009) | (df_lab_events1['ITEMID'] == 50865) | (df_lab_events1['ITEMID'] == 50929) | (df_lab_events1['ITEMID'] == 50997)) & (df_lab_events1['VALUENUM'] > 0).astype('int'))

In [ ]:
print(df_lab_events1.query('ITEMID == 51009 | ITEMID == 50865 | ITEMID == 50929 | ITEMID == 50997').tail(10))

# Re-check the files post the new columns added

In [ ]:
df_chart_events.head(2)

In [ ]:
df_lab_events1.head(2)

In [ ]:
df_lab_events1.query('FLAG == "abnormal"').count()
#df_lab_events1.count()
#f_chart_events.count()

# Write both event files to the csv files

In [ ]:
#from pathlib import Path  
#filepath = Path('C:/Users/Chicki/Desktop/Data/updated')  
#filepath.parent.mkdir(parents=True, exist_ok=True)  

In [ ]:
#df.to_csv ('C:/Users/Chicki/Desktop/Data/updated/ChartEvents_conv.csv', index = None, header=True)

In [ ]:
#df_chart_events.to_csv ('C:/Users/Chicki/Desktop/Data/updated/ChartEvents_conv.csv', index = None, header=True)

In [ ]:
#df_lab_events1.to_csv ('C:/Users/Chicki/Desktop/Data/updated/LabEvents_conv.csv', index = None, header=True)

## SORT THROUGH the CHART EVENTS file to create a SINGLE record for each ADMISSION

In [ ]:
sorted_df_chart_events = df_chart_events.sort_values(by = ['SUBJECT_ID','HADM_ID'], axis = 0, ascending = False, inplace = False)
#sorted_df_chart_events.query('HADM_ID == 115247')
#sorted_df_chart_events.groupby('HADM_ID').nunique()
sorted_df_chart_events.head()

In [ ]:
#print(sorted_df_chart_events['SUBJECT_ID'].loc[sorted_df_chart_events.index[1]])
#dataframe_used = sorted_df_chart_events.head(100)
#dataframe_used.tail()
#dataframe_used.dtypes
#print(dataframe_used.columns.get_loc('SUBJECT_ID'))

In [ ]:
### CREATE the consolidated record for each ADMISSION ID in CHART EVENTS
temp_SUBJECT = 0
temp_HADM = 0
i = 0
j = 0
k = 0
new_chart_events = pd.DataFrame()

## just update last record with a different SUBJECT_ID to check code working
#dataframe_used.at[99,'SUBJECT_ID'] = 69
##
for i, row in sorted_df_chart_events.iterrows():
    
    #if temp_SUBJECT != dataframe_used['SUBJECT_ID'].loc[dataframe_used.index[i]]:
    #if temp_SUBJECT != dataframe_used.iloc[i,0]:
    if temp_SUBJECT == row['SUBJECT_ID']:
        continue                                          
    #temp_SUBJECT = dataframe_used['SUBJECT_ID'].loc[dataframe_used.index[i]]
    #temp_SUBJECT = dataframe_used.iloc[i,0]
    temp_SUBJECT = row['SUBJECT_ID']
    temp_df = sorted_df_chart_events.query('SUBJECT_ID == @temp_SUBJECT')
    #print(temp_df)
    
    for j, row1 in temp_df.iterrows():
        #if temp_HADM != temp_df['HADM_ID'].loc[temp_df.index[j]]:
        if temp_HADM != row1['HADM_ID']: 
            k = k + 1
            temp_HADM = row1['HADM_ID']
            temp_df1 = temp_df.query('HADM_ID == @temp_HADM')
            new_chart_events.at[k,'SUBJECT_ID'] = row1['SUBJECT_ID']
            new_chart_events.at[k,'HADM_ID'] = row1['HADM_ID']
           # new_df.head()
            for col_name in temp_df1[['temphigh','heartratehigh','sysbplow','diasbplow','meanbplow','fio2high','respratehigh','norepinephrine','mechanical_ventilator']]:
                #    new_df.at[k,col_name] = temp_df1.apply(lambda x: True if temp_df1[[x]].all() else False, axis=1)
                    new_chart_events.at[k, col_name] = temp_df1[col_name].any()
                                               
                                          
#print(new_df.head(5))
#print(k)
new_chart_events.count()
    

In [ ]:
#TEST_df = temp_df1.head(5)
#TEST_df['temphigh'] = True
#for col_name in TEST_df[['SUBJECT_ID','temphigh', 'heartratehigh']]:
#    print(TEST_df[col_name].any())
    
#print(TEST_df)

In [ ]:
#Check for the unique values in the main columns we are looking for
for col_name in new_chart_events[['temphigh','heartratehigh','sysbplow','diasbplow','meanbplow','fio2high','respratehigh','norepinephrine','mechanical_ventilator']]:    
    print(new_chart_events[col_name].unique())
    #print(col_name)

temp_SUBJECT = 1
temp_SUBJECT1 = 1
temp_HADM1 = 1

#for i, row in sorted_df_chart_events.head(2).iterrows(): 
    #temp_SUBJECT = row[i]
    #print(f"{row['ITEMID']}")
x = 0    
xx = sorted_df_chart_events.head(5)
#print(xx)
for i, row in xx.iterrows():
    #if temp_SUBJECT == xx.at[i, 'SUBJECT_ID'].all():
    #if temp_SUBJECT == xx['SUBJECT_ID'].iloc[i]
    #if temp_SUBJECT == xx.iloc[[0],['SUBJECT_ID']]:
    if temp_SUBJECT == row['SUBJECT_ID']: 
    #if temp_SUBJECT == xx['SUBJECT_ID'].loc[xx.index[i]]:
        continue                                          
    temp_SUBJECT = row['SUBJECT_ID']
    #temp_SUBJECT == row['SUBJECT_ID']
    #print(temp_SUBJECT)
    temp_df =  xx.query('SUBJECT_ID == @temp_SUBJECT')
    for j, row1 in temp_df.iterrows():
            #print(temp_df.count)
            temp_SUBJECT1 = row1['SUBJECT_ID']
            temp_HADM1 = row1['HADM_ID']
            temp_ITEMID1 = row1['ITEMID']
            #temp_HADM1 = xx['HADM_ID'].loc[xx.index[i]]
            #print(temp_HADM1)
            print(temp_ITEMID1)

In [ ]:
#temp_df = dataframe_used.query('SUBJECT_ID == 99983')
#y = temp_df['HADM_ID'].loc[temp_df.index[1]]

#y = dataframe_used['HADM_ID'].loc[dataframe_used.index[99]]
#y = dataframe_used.iloc[1,1]
#print(y)
#if y != dataframe_used['SUBJECT_ID'].loc[dataframe_used.index[1]]:
 #   print("value not matching:", dataframe_used['SUBJECT_ID'].loc[dataframe_used.index[1]])
#print(y)

In [ ]:
new_chart_events.head(10)

### Write the New Chart Events file to a csv

In [ ]:
new_chart_events.to_csv('C:/Users/Chicki/Desktop/Data/updated/new_chart_events.csv', index = None, header=True)

## SORT THROUGH THE LAB EVENTS FILE TO CREATE A SINGLE RECORD FOR EACH ADMISSION

In [ ]:
sorted_df_lab_events = df_lab_events1.sort_values(by = ['SUBJECT_ID','HADM_ID'], axis = 0, ascending = False, inplace = False)
#sorted_df_lab_events.query('HADM_ID == 115247')
#sorted_df_lab_events.groupby('HADM_ID').nunique()
sorted_df_lab_events.tail()

In [ ]:
# ColNames to be used to create the new dataframe
for col_name in sorted_df_lab_events:    
    #print(new_chart_events[col_name].unique())
    print(col_name)

In [ ]:
### CREATE the consolidated record for each ADMISSION ID in "LAB" EVENTS
temp_SUBJECT = 0
temp_HADM = 0
i = 0
j = 0
k = 0
new_lab_events = pd.DataFrame()

## just update last record with a different SUBJECT_ID to check code working
#dataframe_used.at[99,'SUBJECT_ID'] = 69
##
for i, row in sorted_df_lab_events.iterrows():
    
    #if temp_SUBJECT != dataframe_used['SUBJECT_ID'].loc[dataframe_used.index[i]]:
    #if temp_SUBJECT != dataframe_used.iloc[i,0]:
    if temp_SUBJECT == row['SUBJECT_ID']:
        continue                                          
    #temp_SUBJECT = dataframe_used['SUBJECT_ID'].loc[dataframe_used.index[i]]
    #temp_SUBJECT = dataframe_used.iloc[i,0]
    temp_SUBJECT = row['SUBJECT_ID']
    temp_df = sorted_df_lab_events.query('SUBJECT_ID == @temp_SUBJECT')
    #print(temp_df)
    
    for j, row1 in temp_df.iterrows():
        #if temp_HADM != temp_df['HADM_ID'].loc[temp_df.index[j]]:
        if temp_HADM != row1['HADM_ID']: 
            k = k + 1
            temp_HADM = row1['HADM_ID']
            temp_df1 = temp_df.query('HADM_ID == @temp_HADM')
            new_lab_events.at[k,'SUBJECT_ID'] = row1['SUBJECT_ID']
            new_lab_events.at[k,'HADM_ID'] = row1['HADM_ID']
           # new_df.head()
            for col_name in temp_df1[['spo2high','glucoselow','bicarbonatehigh','bicarbonatelow','creatininehigh','creatininelow','hematocrithigh','hematocritlow','hemoglobinhigh','hemoglobinlow',
                                      'platelethigh','plateletlow','potassiumhigh','potassiumlow','bunhigh','bunlow','wbchigh','wbclow','antibiotics']]:
                #    new_df.at[k,col_name] = temp_df1.apply(lambda x: True if temp_df1[[x]].all() else False, axis=1)
                    new_lab_events.at[k, col_name] = temp_df1[col_name].any()
                                               
                                          
#print(new_df.head(5))
#print(k)
new_lab_events.count()

In [ ]:
#Check for the unique values in the main columns we are looking for
for col_name in new_lab_events[['spo2high','glucoselow','bicarbonatehigh','bicarbonatelow','creatininehigh','creatininelow','hematocrithigh','hematocritlow','hemoglobinhigh','hemoglobinlow',
                                      'platelethigh','plateletlow','potassiumhigh','potassiumlow','bunhigh','bunlow','wbchigh','wbclow','antibiotics']]:
    print(new_lab_events[col_name].unique())
    #print(col_name)

In [ ]:
new_lab_events.head()
#new_lab_events.count()

## Write the New Lab Events file to a csv

In [ ]:
new_lab_events.to_csv('C:/Users/Chicki/Desktop/Data/updated/new_lab_events.csv', index = None, header=True)

## In case the new files are not in current session, read the new files in...

In [3]:
new_chart_events = pd.read_csv('C:/Users/Chicki/Desktop/Data/updated/new_chart_events.csv')

In [4]:
new_lab_events = pd.read_csv('C:/Users/Chicki/Desktop/Data/updated/new_lab_events.csv')

In [5]:
new_chart_events.count()

SUBJECT_ID               10760
HADM_ID                  10760
temphigh                 10760
heartratehigh            10760
sysbplow                 10760
diasbplow                10760
meanbplow                10760
fio2high                 10760
respratehigh             10760
norepinephrine           10760
mechanical_ventilator    10760
dtype: int64

In [6]:
new_lab_events.count()

SUBJECT_ID         19438
HADM_ID            19438
spo2high           19438
glucoselow         19438
bicarbonatehigh    19438
bicarbonatelow     19438
creatininehigh     19438
creatininelow      19438
hematocrithigh     19438
hematocritlow      19438
hemoglobinhigh     19438
hemoglobinlow      19438
platelethigh       19438
plateletlow        19438
potassiumhigh      19438
potassiumlow       19438
bunhigh            19438
bunlow             19438
wbchigh            19438
wbclow             19438
antibiotics        19438
dtype: int64

### JOIN the two EVENTS FILES... FINALLY!!!

In [9]:
df_final_input = pd.merge(new_lab_events, new_chart_events, on = ('SUBJECT_ID','HADM_ID'), how = 'inner')
df_final_input.count()

SUBJECT_ID               7120
HADM_ID                  7120
spo2high                 7120
glucoselow               7120
bicarbonatehigh          7120
bicarbonatelow           7120
creatininehigh           7120
creatininelow            7120
hematocrithigh           7120
hematocritlow            7120
hemoglobinhigh           7120
hemoglobinlow            7120
platelethigh             7120
plateletlow              7120
potassiumhigh            7120
potassiumlow             7120
bunhigh                  7120
bunlow                   7120
wbchigh                  7120
wbclow                   7120
antibiotics              7120
temphigh                 7120
heartratehigh            7120
sysbplow                 7120
diasbplow                7120
meanbplow                7120
fio2high                 7120
respratehigh             7120
norepinephrine           7120
mechanical_ventilator    7120
dtype: int64

In [7]:
new_chart_events.head()

,SUBJECT_ID,HADM_ID,temphigh,heartratehigh,sysbplow,diasbplow,meanbplow,fio2high,respratehigh,norepinephrine,mechanical_ventilator
0,99995.0,137810.0,True,True,False,True,True,False,True,False,False
1,99983.0,117390.0,True,True,True,True,True,False,True,False,False
2,99982.0,183791.0,False,True,True,True,True,False,True,False,False
3,99965.0,101083.0,False,True,False,True,True,False,True,False,False
4,99957.0,148574.0,False,True,False,True,True,False,True,False,False


In [8]:
new_lab_events.head()

,SUBJECT_ID,HADM_ID,spo2high,glucoselow,bicarbonatehigh,bicarbonatelow,creatininehigh,creatininelow,hematocrithigh,hematocritlow,...,hemoglobinlow,platelethigh,plateletlow,potassiumhigh,potassiumlow,bunhigh,bunlow,wbchigh,wbclow,antibiotics
0,99983.0,117390.0,False,False,True,False,True,False,False,True,...,True,False,False,False,False,False,False,False,False,False
1,99957.0,148574.0,False,False,False,False,False,False,False,True,...,False,True,False,False,True,False,False,True,False,False
2,99955.0,108494.0,True,False,False,True,True,False,False,True,...,True,False,True,False,True,True,False,True,False,True
3,99944.0,185654.0,False,False,True,False,True,False,False,True,...,True,False,False,True,True,True,False,True,False,False
4,99939.0,159023.0,True,False,False,True,True,False,False,True,...,True,False,True,True,False,True,False,True,False,False


In [10]:
df_final_input.head()

,SUBJECT_ID,HADM_ID,spo2high,glucoselow,bicarbonatehigh,bicarbonatelow,creatininehigh,creatininelow,hematocrithigh,hematocritlow,...,antibiotics,temphigh,heartratehigh,sysbplow,diasbplow,meanbplow,fio2high,respratehigh,norepinephrine,mechanical_ventilator
0,99983.0,117390.0,False,False,True,False,True,False,False,True,...,False,True,True,True,True,True,False,True,False,False
1,99957.0,148574.0,False,False,False,False,False,False,False,True,...,False,False,True,False,True,True,False,True,False,False
2,99955.0,108494.0,True,False,False,True,True,False,False,True,...,True,True,True,True,True,True,True,True,False,False
3,99944.0,185654.0,False,False,True,False,True,False,False,True,...,False,True,True,False,True,True,True,True,False,False
4,99939.0,159023.0,True,False,False,True,True,False,False,True,...,False,True,True,False,False,False,True,True,False,False


### Drop the unnecessary columns from the final input.. i.e. subject id and hadm id

In [11]:
df_final_input = df_final_input.drop(['SUBJECT_ID', 'HADM_ID'], axis=1)
df_final_input.head()

,spo2high,glucoselow,bicarbonatehigh,bicarbonatelow,creatininehigh,creatininelow,hematocrithigh,hematocritlow,hemoglobinhigh,hemoglobinlow,...,antibiotics,temphigh,heartratehigh,sysbplow,diasbplow,meanbplow,fio2high,respratehigh,norepinephrine,mechanical_ventilator
0,False,False,True,False,True,False,False,True,False,True,...,False,True,True,True,True,True,False,True,False,False
1,False,False,False,False,False,False,False,True,False,False,...,False,False,True,False,True,True,False,True,False,False
2,True,False,False,True,True,False,False,True,False,True,...,True,True,True,True,True,True,True,True,False,False
3,False,False,True,False,True,False,False,True,False,True,...,False,True,True,False,True,True,True,True,False,False
4,True,False,False,True,True,False,False,True,False,True,...,False,True,True,False,False,False,True,True,False,False


In [12]:
### WRITE THE FINAL INPUT FILE OUT.....

df_final_input.to_csv('C:/Users/Chicki/Desktop/Data/updated/final_input.csv', index = None, header=True)

### DATA PREPARATION FOR THE MODEL - TRAIN AND TEST SETS

In [19]:
class MIMICDataset(torch.utils.data.Dataset):
    def __init__(self, X, Y, sequence_lengths):
        super(MIMICDataset, self).__init__()
        self.X = X
        self.Y = Y
        self.sequence_lengths = sequence_lengths

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx], self.sequence_lengths[idx]


def process_mimic_data(feature="wbchigh"):
    feature_names = [
        "temphigh",
        "heartratehigh",
        "sysbplow",
        "diasbplow",
        "meanbplow",
        "spo2high",
        "fio2high",
        "respratelow",
        "glucoselow",
        "bicarbonatehigh",
        "bicarbonatelow",
        "creatininehigh",
        "creatininelow",
        "hematocrithigh",
        "hematocritlow",
        "hemoglobinhigh",
        "hemoglobinlow",
        "platelethigh",
        "plateletlow",
        "potassiumlow",
        "potassiumhigh",
        "bunhigh",
        "bunlow",
        "wbchigh",
        "wbclow",
        "antibiotics",
        "norepinephrine",
        "mechanical_ventilator"
        #"age",
       # "weight",
    ]

    idx = feature_names.index(feature)

#    with open(mimic_root, "rb") as f:
#        MIMIC_data = pickle.load(f)

    MIMIC_data = pd.read_csv('C:/Users/Chicki/Desktop/Data/updated/final_input.csv')
    
    Y = MIMIC_data["longitudinal"][:, :, idx]  # 'wbchigh'
    L = MIMIC_data["trajectory_lengths"]

    return Y, L


def get_mimic_dataset(X, Y, L, idx):
    X_ = torch.nn.utils.rnn.pad_sequence([torch.FloatTensor(X[i]).reshape(-1, 1) for i in idx], batch_first=True)
    Y_ = torch.nn.utils.rnn.pad_sequence(
        [torch.FloatTensor(Y[i]).reshape(-1, 1) for i in idx], batch_first=True
    ).float()

    return MIMICDataset(X_, Y_, [L[i] for i in idx])


def get_mimic_splits(
    n_train=2000, n_calibration=1823, n_test=500, conformal=True, feature="wbchigh", horizon=2, cached=True, seed=None
):
    if seed is None:
        seed = 0
    else:
        cached = False

    if cached:
        if conformal:
            with open("processed_data/mimic_conformal.pkl", "rb") as f:
                train_dataset, calibration_dataset, test_dataset = pickle.load(f)
        else:
            with open("processed_data/mimic_raw.pkl", "rb") as f:
                train_dataset, calibration_dataset, test_dataset = pickle.load(f)

    else:
        perm = np.random.RandomState(seed=seed).permutation(n_train + n_calibration + n_test)
        train_idx = perm[:n_train]
        calibration_idx = perm[n_train : n_train + n_calibration]
        train_calibration_idx = perm[: n_train + n_calibration]
        test_idx = perm[n_train + n_calibration :]

        Y, L = process_mimic_data(feature=feature)
        assert n_train + n_calibration + n_test == len(np.where(L > 4)[0])

        scaler = StandardScaler()
        scaler.fit(Y[np.where(L > 4)[0]][train_idx].reshape(-1, 1))

        X_ = []
        X_unscaled = []
        Y_ = []
        L_ = []
        for k in np.where(L > 4)[0]:
            X_.append(scaler.transform(Y[k, : L[k] - horizon].reshape(-1, 1)).reshape(L[k] - horizon))
            X_unscaled.append(Y[k, : L[k] - horizon])
            Y_.append(Y[k, L[k] - horizon : L[k]])
            L_.append(L[k])

        X_test = [X_unscaled[i] for i in test_idx]
        Y_test = [Y_[i] for i in test_idx]
        with open("processed_data/mimic_test_vis.pkl", "wb") as f:
            pickle.dump((X_test, Y_test), f, protocol=pickle.HIGHEST_PROTOCOL)

        if conformal:
            train_dataset = get_mimic_dataset(X_, Y_, L, train_idx)
            calibration_dataset = get_mimic_dataset(X_, Y_, L, calibration_idx)
            test_dataset = get_mimic_dataset(X_, Y_, L, test_idx)

            with open("processed_data/mimic_conformal.pkl", "wb") as f:
                pickle.dump((train_dataset, calibration_dataset, test_dataset), f, protocol=pickle.HIGHEST_PROTOCOL)

        else:
            train_dataset = (
                [X_[i] for i in train_calibration_idx],
                [Y_[i] for i in train_calibration_idx],
            )
            calibration_dataset = None
            test_dataset = [X_[i] for i in test_idx], [Y_[i] for i in test_idx]

            with open("processed_data/mimic_raw.pkl", "wb") as f:
                pickle.dump((train_dataset, calibration_dataset, test_dataset), f, protocol=pickle.HIGHEST_PROTOCOL)

    return train_dataset, calibration_dataset, test_dataset

print(train_dataset.head())

NameError: name 'train_dataset' is not defined

#  Items

In [ ]:
#df_items = pd.read_csv('C:/Users/Chicki/Desktop/Data/D_ITEMS.csv')
#print((df_items.head(5)))
#print(df_items['ITEMID' == 40055])
#print(df_items.query('ITEMID == 225411' or 'ITEMID = 6185'))
#query_items = df_items.query('ITEMID == 50821')
#query_items = df_items.query('UNITNAME == "%"')
#query_items = df_items.query('ITEMID')
#print(query_items)

In [ ]:
#query_items_specific = df_items.query('LABEL.str.contains("rate", na=False)', engine = 'python', )
#query_items_specific = df_items.query('LABEL.str.contains("antib", na=False)', engine = 'python', )
#print(query_items_specific.head(20))
#df_items['LABEL'].str.contains('sys', na=False)

# Lab Items

In [ ]:
#df_lab_items = pd.read_csv('C:/Users/Chicki/Desktop/Data/D_LABITEMS.csv')
##print((df_lab_items.head(5)))
#query_lab_items = df_lab_items.query('ITEMID == 51006')
#query_lab_events = df_lab_items.query('ontains("Gas", na=False)', engine = 'python')
#query_lab_items = df_lab_items.query('LABEL.str.contains("cin", na=False)', engine = 'python')
#print(query_lab_items)


In [ ]:
#df_micro_events = pd.read_csv('C:/Users/Chicki/Desktop/Data/MICROBIOLOGYEVENTS1.csv', nrows = 9000000)
#print(df_micro_events.tail(5))

## Found no matches between items and lab items

In [ ]:
#df_items_join = pd.merge(df_items, df_lab_items, on = 'ITEMID', how = 'inner')
#df_items_join.count()
#df_items_outer_join = pd.merge(df_items, df_lab_items, on = 'ITEMID', how = 'left')
#df_items_outer_join.count()

# Join of the Chart events and Items

In [ ]:
#df_chart_events_join = pd.merge(df_chart_events1, df_items, on = 'ITEMID', how = 'left')
#df_chart_events_join.count()
#df_chart_events_join.head(5)
#df_chart_events_join.isnull().sum()
#query_chart_events_join = df_chart_events_join.query('ABBREVIATION.str.contains("weig", na=False)', engine = 'python')
#query_chart_events_join = df_chart_events_join.query('ITEMID == 225411')
#df_chart_events_join.sort_values(by=['HADM_ID']).head(5)
#print(query_chart_events_join)

In [ ]:
#df_items.isnull().sum()
#df_chart_events_join.count()

In [ ]:
#query1_chart_events_join = df_chart_events_join.query('ITEMID == 50821', engine = 'python', )
#query1_chart_events_join = df_chart_events_join.query('ITEMID == 223835', engine = 'python')
#query1_chart_events_join = df_chart_events_join.query('LABEL.str.contains("wei", na=False)', engine = 'python', )
#query1_chart_events_join = df_chart_events_join.query('VALUEUOM.str.contains("%", na=False)', engine = 'python', )
#query1_chart_events_join.head(5)

# Join of the Lab events and Lab items

In [ ]:
new_chart_events.count()
#df_lab_events_join = pd.merge(df_lab_events, df_lab_items, on = 'ITEMID', how = 'left')
#df_lab_events_join.count()
#df_lab_events_join.head(5)
#df_lab_events_join.isnull().sum()

In [ ]:
#query_lab_events_join = df_lab_events_join.query('FLAG == "abnormal" & VALUEUOM.str.contains("bp", na=False)', engine = 'python')
#query_lab_events_join = df_lab_events_join.query('FLAG == "abnormal"', engine = 'python')
#query_lab_events_join = df_lab_events_join.query('ITEMID == 50821')
#query_lab_events_join = df_lab_events_join.query('LABEL.str.contains("rate", na=False)', engine = 'python')
#query_lab_events_join = df_lab_events_join.query('VALUEUOM.str.contains("bp", na=False)', engine = 'python')
#query_lab_events_join.head(5)

In [ ]:
#WBC ITEMID is 51301. Flag 'Abnormal' shows the wbc is high
#query1_lab_events_join = df_lab_events_join.query('ITEMID == 50821')
#query1_lab_events_join.count()
#query1_lab_events_join = df_lab_events_join.query('LABEL.str.contains("temp", na=False)', engine = 'python', )
#query1_lab_events_join.tail(10)

# Check for admission records 

In [ ]:
#query1_lab_events_join = df_lab_events_join.query('HADM_ID == 148595')
#query1_lab_events_join.tail(10)

In [ ]:
#query1_chart_events_join = df_lab_events_join.query('HADM_ID == 148595')
#query1_chart_events_join.tail(10)

# Join the Chart and Lab events

# just trials first

In [ ]:
#trial_chart = df_chart_events1.head(10000)
#trial_lab = df_lab_events1.head(10000)
#trial_join = pd.merge(trial_chart, trial_lab, on = 'HADM_ID', how = 'inner')
trial_join.count()
#trial_join.head(5)

In [ ]:
#trial_join1 = pd.merge(trial_chart, trial_lab, on = 'HADM_ID', how = 'outer')
#trial_join1.count()
#trial_join1.head(5)

In [ ]:
#trial_join2 = pd.merge(trial_chart, trial_lab, on = 'HADM_ID', how = 'right')
#trial_join2.count()
#print(trial_join2.query('HADM_ID == 165660'))
#trial_join2.isna().sum()  -- to find the NA values in each column

In [ ]:
#df_lab_events1.count()

In [ ]:
#df_chart_events1.count()

In [ ]:
#df_join_chart_lab_events = pd.merge(df_chart_events1, df_lab_events1, on = ['SUBJECT_ID','HADM_ID'], how = 'inner', sort=False)
#df_join_chart_lab_events.count()

In [ ]:
#df_datetime_events = pd.read_csv('C:/Users/Chicki/Desktop/Data/DATETIMEEVENTS.csv', low_memory = False)
#query_datetime_events = df_datetime_events.query('WARNING == 1 & VALUEUOM != "DATE" & VALUEUOM != "Date"', engine = 'python')
#query_datetime_events = df_datetime_events.query('ITEMID == 3112')
#print(query_datetime_events.head(15))
#print(df_datetime_events.head(10))
#df_datetime_events.count()


In [ ]:
#df_proc_events = pd.read_csv('C:/Users/Chicki/Desktop/Data/PROCEDUREEVENTS_MV1.csv')
#query_proc_events = df_lab_events_join.query('ITEMID == 3112')
#print(query_proc_events)
#df_proc_events.count()
#df_proc_events.head(5)

In [ ]:
#df_note_events = pd.read_csv('C:/Users/Chicki/Desktop/Data/NOTEEVENTS1.csv', nrows=10000000, low_memory=False )
#df_note_events.count()
#df_note_events.tail(5)
#query_df_note_events = df_note_events.query('TEXT.str.contains("weight", na=False)', engine = 'python')
#query_df_note_events.head(5)

In [ ]:
#df_cpt_events = pd.read_csv('C:/Users/Chicki/Desktop/Data/CPTEVENTS1.csv', low_memory=False)
#df_cpt_events.count()
#df_cpt_events.tail(5)
#query_cpt_events = df_cpt_events.query('DESCRIPTION.str.contains("weig", na=False)', engine = 'python')
#query_cpt_events = df_cpt_events.query('CPT_NUMBER == 94004')
#query_cpt_events = df_cpt_events.query('CPT_NUMBER > 94000 & DESCRIPTION.str.contains("VENT", na=False)', engine = 'python')
#query_cpt_events.head(20)